In [1]:
import pandas as pd
import twitterscraper
from bs4 import BeautifulSoup
import csv
import datetime
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re

INFO: {'User-Agent': 'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16'}


# Positive Case EDA

In [24]:
bots_df = pd.read_pickle('bots.pkl')

In [25]:
bots_df = bots_df[bots_df['language']=='English']

In [26]:
bots_df.shape

(952915, 21)

In [28]:
len(bots_df['external_author_id'].unique())

711

In [6]:
sum(bots_df['post_type'].isna())

153400

In [7]:
bots_df['post_type'].value_counts()

RETWEET        114713
QUOTE_TWEET      3796
Name: post_type, dtype: int64

In [8]:
# remove idenitfying characteristics from five thirty eight
bots_df.drop(['harvested_date','account_type','account_category','article_url'], axis = 1, inplace= True)

In [9]:
bots_df = bots_df[['external_author_id','author', 'content', 'publish_date', 'following', 'followers', 'updates', 'retweet','new_june_2018','post_type','tco1_step1']]

In [10]:
bots_df.rename({'external_author_id':'user_id','author':'screen_name',
           'publish_date':'timestamp','content':'text','tco1_step1':'links','retweet':'retweets'}, axis = 1, inplace = True)

In [11]:
bots_df['hashtags'] = bots_df['text'].str.findall(r'\B(\#[a-zA-Z]+\b)(?!;)')
# bots_df['hashtags'] = [','.join(map(str, l)) for l in bots_df['hashtags']]

In [12]:
bots_df['timestamp'] = pd.to_datetime(bots_df['timestamp'])

In [13]:
bots_account_df = bots_df.groupby('user_id').agg({'timestamp':['min', 'max'],
                                                     'text':'count'}).reset_index()

In [14]:
bots_account_df.columns = ['user_id','date_joined','max_date','posts']

In [15]:
bots_account_df['active_period'] = bots_account_df['max_date'] - bots_account_df['date_joined']
bots_account_df['active_period'] = bots_account_df['active_period'].dt.days

In [16]:
bots_df = bots_df.merge(bots_account_df, how = 'left', on = 'user_id')

In [17]:
bots_df['post_type'].value_counts()

RETWEET        114713
QUOTE_TWEET      3796
Name: post_type, dtype: int64

In [18]:
bots_df['is_quote_tweet'] = bots_df['post_type']=='QUOTE_TWEET'
bots_df['is_retweet'] = bots_df['post_type']=='RETWEET'

In [19]:
bots_df.head()

,user_id,screen_name,text,timestamp,following,followers,updates,retweets,new_june_2018,post_type,links,hashtags,date_joined,max_date,posts,active_period,is_quote_tweet,is_retweet
0,890000000000000000,JOHAAJOTS,"#jobb According to Mad Max, no one is More “DE...",2017-08-05 11:41:00,2960,669,400,0,0,NaN,https://twitter.com/johaajots/status/893799058...,"[#jobb, #job]",2017-07-25 08:52:00,2017-08-18 22:37:00,14046,24,False,False
1,890000000000000000,JOHAAJOTS,#jobb TRUMP EFFECT : July Job Numbers CRUSH Ex...,2017-08-05 11:46:00,2960,669,401,0,0,NaN,https://twitter.com/johaajots/status/893800394...,"[#jobb, #job]",2017-07-25 08:52:00,2017-08-18 22:37:00,14046,24,False,False
2,890000000000000000,JOHAAJOTS,#jobb You Won’t Believe Who is Now Calling for...,2017-08-05 11:46:00,2960,669,402,0,0,NaN,https://twitter.com/johaajots/status/893800402...,"[#jobb, #job]",2017-07-25 08:52:00,2017-08-18 22:37:00,14046,24,False,False
3,890000000000000000,JOHAAJOTS,#jobb Fake News Media Ignoring This! https://t...,2017-08-05 11:51:00,2960,669,406,0,0,NaN,https://twitter.com/johaajots/status/893801597...,"[#jobb, #job]",2017-07-25 08:52:00,2017-08-18 22:37:00,14046,24,False,False
4,890000000000000000,JOHAAJOTS,#jobb WITCH HUNT : Russia Probe Includes Tenan...,2017-08-05 11:51:00,2960,669,404,0,0,NaN,https://twitter.com/johaajots/status/893801584...,"[#jobb, #job]",2017-07-25 08:52:00,2017-08-18 22:37:00,14046,24,False,False


In [20]:
bots_df['is_retweet'] = bots_df['text'].str.startswith('https://twitter.com', na = False)
bots_df['is_quote_tweet'] = (bots_df['links'].str.contains('https://twitter.com', na = False, regex= False)) & (~bots_df['is_retweet'])

In [23]:
# bots_df['is_bot']=1

# common_cols = df.columns.intersection(bots_df.columns)
# bots_df = bots_df[common_cols]

# bots_df['hashtags'].fillna(0, inplace= True)
# bots_df['links'].fillna('', inplace= True)

# bots_df.to_pickle('validation_bots.pkl')

# Merging

In [29]:
df = pd.read_pickle('verified_cleaned.pkl')

In [30]:
df.head()

,user_id,username,screen_name,timestamp,text,hashtags,retweets,likes,has_media,links,is_reply_to,is_replied,parent_tweet_id,date_joined,followers,following,links_string,is_retweet,is_quote_tweet
6,3519500234,State of Arizona Research Library,StateLibAZ,2018-12-19 14:13:16,A blast from our DAZL past! Make sure you are ...,[],0,0,True,https://www.facebook.com/digitalazlibrary/phot...,False,False,,NaN,NaN,NaN,https://www.facebook.com/digitalazlibrary/phot...,False,False
15,104904750,Michele Reagan,SecretaryReagan,2018-07-30 15:26:05,The @StateLibAZ had a busy year consolidating ...,[],3,0,False,http://go.azsos.gov/g7p7,False,False,,NaN,NaN,NaN,http://go.azsos.gov/g7p7,False,False
31,3519500234,State of Arizona Research Library,StateLibAZ,2018-04-04 18:22:03,From upcoming conferences and workshops to sup...,[],1,0,False,https://conta.cc/2GBmbdS,False,False,,NaN,NaN,NaN,https://conta.cc/2GBmbdS,False,False
33,104904750,Michele Reagan,SecretaryReagan,2018-02-06 19:10:03,E-rate deadline approaching http://www.wmicent...,[],1,0,False,http://www.wmicentral.com/news/apache_county/e...,False,False,,NaN,NaN,NaN,http://www.wmicentral.com/news/apache_county/e...,False,False
34,104904750,Michele Reagan,SecretaryReagan,2018-01-09 22:06:23,LIBRARIANS: The January edition of our Library...,[],1,0,True,http://conta.cc/2FlHWKz,False,False,,NaN,NaN,NaN,http://conta.cc/2FlHWKz,False,False


In [32]:
df.shape

(672316, 19)

In [31]:
len(df['user_id'].unique())

75329

In [23]:
# Merging

df['is_bot']=0
bots_df['is_bot']=1

common_cols = df.columns.intersection(bots_df.columns)
A = df[common_cols]
B = bots_df[common_cols]

merged_df = pd.concat([A,B])

In [24]:
del(A,B)

merged_df.sample(10)

,user_id,screen_name,timestamp,text,hashtags,retweets,links,date_joined,followers,following,is_retweet,is_quote_tweet,is_bot
507978,2491401030,CLIFTONHUGHES_,2016-01-10 21:55:00,Tonight on @60Minutes see our client Anthony R...,[],1,http://www.cbsnews.com/live/video/60-minutes-p...,2015-01-13 12:23:00,141.0,405.0,False,False,1
789111,2540611879,ERICARUTTER,2017-01-30 12:09:00,Touched by D'Angelo #AddABandToATVShow,[#AddABandToATVShow],0,NaN,2014-11-27 08:35:00,2257.0,2784.0,False,False,1
111836,35261904,saradiamond,2017-11-17 17:49:35,Thanks friend!! https://twitter.com/jasonthesc...,[],1,https://twitter.com/jasonthescott/status/93157...,10:26 AM - 25 Apr 2009,7528.0,726.0,False,True,0
391867,130557513,mehdirhasan,2018-11-29 15:34:16,The far-right & pro-Likud brigade are up in ar...,[],83,,9:57 AM - 7 Apr 2010,808517.0,2649.0,False,False,0
432556,898000000000000000,CHARMEESTRS,2017-09-19 11:58:00,Liberals Freak Out Over Sean Spicer’s Emmys Jo...,[],0,http://ift.tt/2fgiiyU,2017-08-18 05:52:00,1686.0,4818.0,False,False,1
607896,2495567768,DAILYSANFRAN,2017-02-11 17:13:00,VIDEO: Giant’s ‘Fan Fest’ underway at AT&amp;T...,[],0,https://twitter.com/DailySanFran/status/830464...,2015-12-09 22:38:00,16797.0,4568.0,False,True,1
5579,45850426,brimjar,2016-05-23 17:53:49,Landing in London #aviation #timelapse #boeing...,"[aviation, timelapse, boeing777, widebody]",4,https://www.instagram.com/p/BFwkRRgk-oh/,7:40 AM - 9 Jun 2009,7621.0,275.0,False,False,0
355141,48459607,rubarrichello,2018-07-05 06:59:04,https://www.instagram.com/p/Bk1z0enhcb5/?utm_...,[],2,https://www.instagram.com/p/Bk1z0enhcb5/?utm_s...,12:29 PM - 18 Jun 2009,2086920.0,387.0,False,False,0
397087,1629828379,CASSISHERE,2015-12-07 07:53:00,Up next at 12A PST \| 3A EST \| 8A GMT @hasht...,[],1,NaN,2015-01-06 14:37:00,816.0,1196.0,False,False,1
61105,791956034579345408,comicreliefsch,2017-02-08 16:21:40,COMPETITION: Win a celeb visit to your school ...,[],5,https://goo.gl/mYRsDd,3:53 AM - 28 Oct 2016,4989.0,3456.0,False,False,0


In [25]:
merged_df['is_bot'].value_counts()

merged_df['hashtags'].fillna(0, inplace= True)
merged_df['links'].fillna('', inplace= True)

merged_df[merged_df.isnull().any(axis=1)]
merged_df.to_csv('merged_sample.csv')